In [1]:
import os
import glob
import numpy as np
import pandas as pd

from kolmov.core.base_exporter import export_onnx_tool

Welcome to JupyROOT 6.22/00
Using all sub packages with ROOT dependence


In [2]:
base_path     = os.environ['DATA_PATH']
tunes_path    = os.environ['TUNES_PATH']
analysis_path = os.environ['ANALYSIS_PATH']
print('Variables path defined \n base: %s \n tunes: %s \n analysis: %s' %(base_path,
                                                                          tunes_path,
                                                                          analysis_path))

Variables path defined 
 base: /home/micael/Documents/NeuralRinger/jpsiee_data 
 tunes: /home/micael/Documents/NeuralRinger/jpsiee_tunes 
 analysis: /home/micael/Documents/NeuralRinger/jpsiee_analysis


In [3]:
filename   = 'selected_neurons_info_table.csv' 
tunings_df = pd.read_csv(os.path.join(analysis_path, 'v1.data17/files/%s' %filename))
tunings_df.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,max_sp_val,max_sp_pd_val,...,tight_pd_val_total,tight_fa_val_total,tight_pd_op_passed,tight_fa_op_passed,tight_pd_op_total,tight_fa_op_total,tight_op_threshold,medium_op_threshold,loose_op_threshold,vloose_op_threshold
0,v1.data17.mlp2,0,0,0,0,1,/home/micael/Documents/NeuralRinger/jpsiee_tun...,0,0.946916,0.951160,...,2846,20500,27987,23933,28455,205005,-0.892544,-0.892544,-0.852873,-0.938755
1,v1.data17.mlp2,0,0,0,1,3,/home/micael/Documents/NeuralRinger/jpsiee_tun...,0,0.944001,0.956079,...,2846,20500,27987,25498,28455,205005,-0.844698,-0.844698,-0.807455,-0.885452
2,v1.data17.mlp2,0,0,0,2,1,/home/micael/Documents/NeuralRinger/jpsiee_tun...,0,0.941700,0.946240,...,2846,20500,27987,25581,28455,205005,-0.842062,-0.842062,-0.806970,-0.881531
3,v1.data17.mlp2,0,0,0,3,4,/home/micael/Documents/NeuralRinger/jpsiee_tun...,0,0.944651,0.968377,...,2846,20500,27987,24161,28455,205005,-0.885199,-0.885199,-0.845055,-0.928306
4,v1.data17.mlp2,0,0,0,4,4,/home/micael/Documents/NeuralRinger/jpsiee_tun...,0,0.944673,0.960998,...,2846,20500,27987,25467,28455,205005,-0.883971,-0.883971,-0.848430,-0.918506


# Create a operation dataframe

We need to filter all the models and get the models that should the exported.

In [4]:
# choose a specific model
m_tag = 'v1.data17.mlp2'

# filter the dataframe
tunings_df = tunings_df[tunings_df.train_tag == m_tag]

print('Dataframe size: %i' %(len(tunings_df)))

tunings_df.head()

Dataframe size: 150


,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,max_sp_val,max_sp_pd_val,...,tight_pd_val_total,tight_fa_val_total,tight_pd_op_passed,tight_fa_op_passed,tight_pd_op_total,tight_fa_op_total,tight_op_threshold,medium_op_threshold,loose_op_threshold,vloose_op_threshold
0,v1.data17.mlp2,0,0,0,0,1,/home/micael/Documents/NeuralRinger/jpsiee_tun...,0,0.946916,0.951160,...,2846,20500,27987,23933,28455,205005,-0.892544,-0.892544,-0.852873,-0.938755
1,v1.data17.mlp2,0,0,0,1,3,/home/micael/Documents/NeuralRinger/jpsiee_tun...,0,0.944001,0.956079,...,2846,20500,27987,25498,28455,205005,-0.844698,-0.844698,-0.807455,-0.885452
2,v1.data17.mlp2,0,0,0,2,1,/home/micael/Documents/NeuralRinger/jpsiee_tun...,0,0.941700,0.946240,...,2846,20500,27987,25581,28455,205005,-0.842062,-0.842062,-0.806970,-0.881531
3,v1.data17.mlp2,0,0,0,3,4,/home/micael/Documents/NeuralRinger/jpsiee_tun...,0,0.944651,0.968377,...,2846,20500,27987,24161,28455,205005,-0.885199,-0.885199,-0.845055,-0.928306
4,v1.data17.mlp2,0,0,0,4,4,/home/micael/Documents/NeuralRinger/jpsiee_tun...,0,0.944673,0.960998,...,2846,20500,27987,25467,28455,205005,-0.883971,-0.883971,-0.848430,-0.918506


In [5]:
# now filter to select the operation models
op_key = 'max_sp_op'

# filter the Dataframe
tunings_df = tunings_df.loc[tunings_df.groupby(['et_bin',
                                                'eta_bin',
                                                'model_idx'])[op_key].idxmax(), :]

print('Dataframe size: %i' %(len(tunings_df)))

tunings_df.head()

Dataframe size: 15


,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,max_sp_val,max_sp_pd_val,...,tight_pd_val_total,tight_fa_val_total,tight_pd_op_passed,tight_fa_op_passed,tight_pd_op_total,tight_fa_op_total,tight_op_threshold,medium_op_threshold,loose_op_threshold,vloose_op_threshold
7,v1.data17.mlp2,0,0,0,7,0,/home/micael/Documents/NeuralRinger/jpsiee_tun...,0,0.947722,0.965905,...,2845,20501,27987,24848,28455,205005,-0.912129,-0.912129,-0.870464,-0.946801
32,v1.data17.mlp2,0,1,0,2,4,/home/micael/Documents/NeuralRinger/jpsiee_tun...,0,0.934799,0.956522,...,874,17133,8363,17527,8736,171335,-0.906193,-0.906193,-0.949574,-0.962763
68,v1.data17.mlp2,0,2,0,8,0,/home/micael/Documents/NeuralRinger/jpsiee_tun...,0,0.893922,0.900000,...,50,2711,419,2865,499,27116,-0.909357,-0.916594,-0.923595,-0.925543
94,v1.data17.mlp2,0,3,0,4,3,/home/micael/Documents/NeuralRinger/jpsiee_tun...,0,0.919756,0.954955,...,666,22948,6217,24309,6656,229486,-0.921942,-0.929186,-0.929186,-0.930607
127,v1.data17.mlp2,0,4,0,7,0,/home/micael/Documents/NeuralRinger/jpsiee_tun...,0,0.949865,1.000000,...,27,1667,259,2928,272,16670,-0.927055,-0.927055,-0.924014,-0.927055


In [6]:
tunings_df.file_name.iloc[0]

'/home/micael/Documents/NeuralRinger/jpsiee_tunes/data17_tuned_files/user.mverissi.task.data17_13TeV.1.AllPeriods.sgn.probes_lhmedium_EGAM2.bkg.VProbes_EGAM7.et0_eta0.v1/job_configId_89/tunedDiscr.jobID_0035.pic.gz'

In [7]:
# save the Dataframe
op_filename = '%s_operation_df.csv' %(m_tag)
tunings_df.to_csv(os.path.join(analysis_path, 'v1.data17/files/%s' %op_filename),
                  index=False)

# Test the exporter

Now we have the operation models in a .csv file

In [8]:
# open the op dataframe
m_tag                = 'v1.data17.mlp2'
m_version            = 'v1'
m_model_format_name  = 'data17_13TeV_EGAM2_probes_lhmedium_EGAM7_vetolhvloose.model_v1.electron%s.et%d_eta%d.onnx'
m_output_conf_name   = 'ElectronRinger%sTriggerConfig.conf'
m_tuning_name        = 'TrigL2_20200805_v1'
filename             = '%s_operation_df.csv' %(m_tag)
op_df_path           = os.path.join(analysis_path, 'v1.data17/files/%s' %filename)
export_path          = os.path.join(analysis_path, 'v1.data17/exported_tunings/TrigL2_20200805_mlp2.v1')

In [9]:
m_export_tool = export_onnx_tool(operation_dataframe=op_df_path)

In [10]:
for ioperation in ['Tight', 'Medium', 'Loose', 'VeryLoose']:
    print('Exporting tunings | Operation Point: %s ' %ioperation)
    m_export_tool.create_config_files(operation_point=ioperation, 
                                      tuning_name=m_tuning_name,
                                      version=m_version,
                                      model_format_name=os.path.join(export_path, m_model_format_name),
                                      output_config_name=os.path.join(export_path, m_output_conf_name))
    print('\n')

dense_2 (Dense)              (None, 2)                 202       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3         
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________
None
Processing et bin: 2 | eta bin: 1
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 2)                 202       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3         
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________
None
Processing et bin: 2 | eta bin: 2
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimizati